In [ ]:
import requests
import json
from rdflib import Graph, Namespace

In [ ]:
# get token from FDP server

URL = "http://his-fdp-srv-t1.darwinproject.be:8080/"
URL_token = URL+"tokens"


data='{"email":"XXX","password":"XXX"}'

headers={
    'Content-type':'application/json', 
    'Accept':'application/json'
}

response=requests.post(url=URL_token, data=data, headers=headers)
token_data = json.loads(response.text) # Parses the JSON response text from the server into a Python dictionary.
token = token_data["token"]    
print(token)

the code below is to create a catalogue, in the res=requests.post(url=URL_catalog, data=dcat_catalog_data, headers=headers) the dcat_catalog_data would be the info of the catalogue we take from the view+ttl_templating

In [ ]:
# POST catalog metadata record into fdp server using the token and the rdf content

URL_catalog = URL+"catalog" # Constructs the full URL to the catalog endpoint where the new catalog metadata will be posted.
print("URL catalog:", URL_catalog)

headers={
    'Authorization':f'Bearer {token}',
    'Content-type':'text/turtle' # specify that the rdf is sent in turtle serialisation
}

res=requests.post(url=URL_catalog, data=dcat_catalog_data, headers=headers)

print("Response status code:", res.status_code) # There are multiple type of code status. See here https://restfulapi.net/http-status-codes/
print("Response headers:", res.headers)
print("Response content:", res.text)
#print("Response option:", res.json()) 
#res_option = requests.options(url=URL_catalog, headers=headers)
#print("Allowed methods:", res_option.headers.get('Allow'))

# Extract the catalogue ID from the Location header
if res.status_code == 201:  # Created
    catalog_url = res.headers.get('Location')
    if catalog_url:
        catalogue_id = catalog_url.split('/')[-1]  # Extract the last part of the URL, which is the ID
        print(f"Extracted Catalogue ID: {catalogue_id}")
    else:
        print("No Location header found in the response.")
else:
    print(f"Failed to create catalog. Status code: {res.status_code}")
    
print("done")


The below is used to make the just created catalog from "draft" to "Published"

In [ ]:
URL_pub=URL/+catalogue_id+'/meta/state'
# The /meta/state path is appended to the URL, which is typically the endpoint that controls the state of a catalog (such as DRAFT, PUBLISHED, etc.).

# print(URL)

headers={
    'Authorization': f'Bearer {token}',
    'Accept':'application/json',
    'Content-type':'application/json'
}

in_values="{\"current\":\"PUBLISHED\"}"

res=requests.put(URL_pub, data=in_values, headers=headers)
print("Response status code:", res.status_code)
print("Response content:", res.text)

The below instead is to create a dataset inside the catalog

In [ ]:
right_path = URL+'/dataset' # you don't need to specify the catalog id in the URL bu only the dataset endpoint since the catalog id is deducted from the dataset metadata. Specifically from the dct:isPartOf property.

# Define dataset details in Turtle format
dataset_clo1 = f'@prefix dcat: <http://www.w3.org/ns/dcat#> .\
@prefix dct: <http://purl.org/dc/terms/> .\
@prefix foaf: <http://xmlns.com/foaf/0.1/> .\
<> a dcat:Dataset ;\
dct:title "dataset_Claudio_test1" ;\
dcterms:language <http://id.loc.gov/vocabulary/iso639-1/en> ;\
dct:hasVersion "1.0" ;\
dct:publisher [ a foaf:Agent ; foaf:name "Claudio" ] ;\
dcat:theme <http://blabla.com>;\
dct:isPartOf <http://10.5.8.138:8080/catalog/{catalogue_id}> .'

# Correct headers with the Authorization token
headers = {
    'Authorization': f'Bearer {token}',
    'Content-type': 'text/turtle'  # Specify that the RDF is sent in Turtle serialization
}

# Send a POST request to create a new dataset inside the specific catalog
res = requests.post(url=right_path, data=dataset_clo1, headers=headers)
# put --> http://10.5.8.138:8080/dataset/f19bafa6-a58b-4e11-b504-55a613f000cc

# Check the response from the server
print("Response status code:", res.status_code)  # Should return 201 if successful
print("Response headers:", res.headers)
print("Response content:", res.text)

# Extract the dataset ID from the Location header
if res.status_code == 201:  # Created
    dataset_url = res.headers.get('Location')
    if dataset_url:
        dataset_id = dataset_url.split('/')[-1]  # Extract the last part of the URL, which is the ID
        print(f"Extracted Dataset ID: {dataset_id}")
    else:
        print("No Location header found in the response.")
else:
    print(f"Failed to create dataset. Status code: {res.status_code}")

below is to make the just created dataset from "draft" to "published"

In [ ]:
dataset_id
URL_pubDataset=URL+'/dataset/'+dataset_id+'/meta/state'

print(URL)

headers={
    'Authorization': f'Bearer {token}',
    'Accept':'application/json',
    'Content-type':'application/json'
}

in_values="{\"current\":\"PUBLISHED\"}"

res=requests.put(URL_pubDataset, data=in_values, headers=headers)

The below is a delete operation used to delete the catalog created

In [ ]:
# Define the URL of the catalogue to delete
catalogue_id 
URL_delete_catalog = f"{URL}/catalog/{catalogue_id}"

# Headers with Authorization token
headers = {
    'Authorization': f'Bearer {token}',
    'Accept': 'application/json'
}

# Send a DELETE request to remove the catalog
res = requests.delete(url=URL_delete_catalog, headers=headers)

# Check the response from the server
print("Response status code:", res.status_code)  # Should return 204 if successful
print("Response content:", res.text)


The below is a delete operation used to delete the dataset created

In [ ]:
# Define the URL of the dataset to delete
dataset_id 
URL_delete_dataset = f"http://his-fdp-srv-t1.darwinproject.be:8080/dataset/{dataset_id}"

# Headers with Authorization token
headers = {
    'Authorization': f'Bearer {token}',
    'Accept': 'application/json'
}

# Send a DELETE request to remove the dataset
res = requests.delete(url=URL_delete_dataset, headers=headers)

# Check the response from the server
print("Response status code:", res.status_code)  # Should return 204 if successful
print("Response content:", res.text)

The below is a modify operation used to modify the catalog created

In [ ]:
# Define the URL of the catalogue to modify
catalogue_id 
URL_modify_catalog = f"{URL}/catalog/{catalogue_id}"

# Define updated catalog metadata in Turtle format
updated_catalog_data = 'HERE WE PUT THE UPDATED RDF DATA'

# Headers with Authorization token and content type
headers = {
    'Authorization': f'Bearer {token}',
    'Content-type': 'text/turtle'
}

# Send a PUT request to update the catalog metadata
res = requests.put(url=URL_modify_catalog, data=updated_catalog_data, headers=headers)

# Check the response from the server
print("Response status code:", res.status_code)  # Should return 200 if successful
print("Response content:", res.text)


The below is a modify operation used to modify the dataset created

In [ ]:
# Define the URL of the dataset to modify
dataset_id 
URL_modify_dataset = f"http://his-fdp-srv-t1.darwinproject.be:8080/dataset/{dataset_id}"

# Define updated dataset metadata in Turtle format
updated_dataset_data = f'@prefix dcat: <http://www.w3.org/ns/dcat#> .\
@prefix dct: <http://purl.org/dc/terms/> .\
@prefix foaf: <http://xmlns.com/foaf/0.1/> .\
<> a dcat:Dataset ;\
dct:title "updated_dataset_Claudio_test3" ;\
dcterms:language <http://id.loc.gov/vocabulary/iso639-1/en> ;\
dct:hasVersion "2.0" ;\
dct:publisher [ a foaf:Agent ; foaf:name "Claudio Updated" ] ;\
dcat:theme <http://blabla.com>;\
dct:isPartOf <http://10.5.8.138:8080/catalog/{catalogue_id}> .'

# Headers with Authorization token and content type
headers = {
    'Authorization': f'Bearer {token}',
    'Content-type': 'text/turtle'
}

# Send a PUT request to update the dataset metadata
res = requests.put(url=URL_modify_dataset, data=updated_dataset_data, headers=headers)

# Check the response from the server
print("Response status code:", res.status_code)  # Should return 200 if successful
print("Response content:", res.text)

BELOW ARE THE FUNCTIONS CREATED FROM THE ABOVE CHUNKES OF CODE :*

In [ ]:
# Base URL and token retrieval function
FDP_BASE_URL = "http://his-fdp-srv-t1.darwinproject.be:8080"

def get_fdp_token(email: str, password: str) -> Optional[str]:
    """
    Authenticate with the FDP server and return a token.
    """
    URL_token = f"{FDP_BASE_URL}/tokens"
    data = json.dumps({"email": email, "password": password})
    headers = {
        'Content-type': 'application/json',
        'Accept': 'application/json'
    }
    response = requests.post(url=URL_token, data=data, headers=headers)
    
    if response.status_code == 200:
        token_data = response.json()
        return token_data.get("token")
    else:
        print(f"Failed to get token. Status code: {response.status_code}, Message: {response.text}")
        return None

def create_catalogue(token: str, rdf_content: str) -> Optional[str]:
    """
    Create a catalogue on the FDP server and return the catalogue ID.
    """
    URL_catalog = f"{FDP_BASE_URL}/catalog"
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-type': 'text/turtle'
    }
    response = requests.post(url=URL_catalog, data=rdf_content, headers=headers)
    
    if response.status_code == 201:
        catalog_url = response.headers.get('Location')
        catalogue_id = catalog_url.split('/')[-1] if catalog_url else None
        return catalogue_id
    else:
        print(f"Failed to create catalog. Status code: {response.status_code}, Message: {response.text}")
        return None

def publish_catalogue(token: str, catalogue_id: str) -> bool:
    """
    Publish a catalogue on the FDP server by changing its state to "PUBLISHED".
    """
    URL_pub = f"{FDP_BASE_URL}/{catalogue_id}/meta/state"
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-type': 'application/json'
    }
    data = json.dumps({"current": "PUBLISHED"})
    response = requests.put(URL_pub, data=data, headers=headers)
    
    return response.status_code == 200

def create_dataset(token: str, rdf_content: str) -> Optional[str]:
    """
    Create a dataset on the FDP server and return the dataset ID.
    """
    URL_dataset = f"{FDP_BASE_URL}/dataset"
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-type': 'text/turtle'
    }
    response = requests.post(url=URL_dataset, data=rdf_content, headers=headers)
    
    if response.status_code == 201:
        dataset_url = response.headers.get('Location')
        dataset_id = dataset_url.split('/')[-1] if dataset_url else None
        return dataset_id
    else:
        print(f"Failed to create dataset. Status code: {response.status_code}, Message: {response.text}")
        return None

def publish_dataset(token, dataset_id, fdp_url):
    """Publishes a dataset by changing its state to 'PUBLISHED'."""
    url = f"{fdp_url}/dataset/{dataset_id}/meta/state"
    headers = {
        'Authorization': f'Bearer {token}',
        'Accept': 'application/json',
        'Content-type': 'application/json'
    }
    data = '{"current":"PUBLISHED"}'
    response = requests.put(url, data=data, headers=headers)
    return response.status_code, response.text

def delete_catalogue(token, catalogue_id, fdp_url):
    """Deletes a catalogue from the FDP server."""
    url = f"{fdp_url}/catalog/{catalogue_id}"
    headers = {
        'Authorization': f'Bearer {token}',
        'Accept': 'application/json'
    }
    response = requests.delete(url=url, headers=headers)
    return response.status_code, response.text

def delete_dataset(token, dataset_id, fdp_url):
    """Deletes a dataset from the FDP server."""
    url = f"{fdp_url}/dataset/{dataset_id}"
    headers = {
        'Authorization': f'Bearer {token}',
        'Accept': 'application/json'
    }
    response = requests.delete(url=url, headers=headers)
    return response.status_code, response.text

def modify_catalogue(token, catalogue_id, fdp_url, updated_catalog_data):
    """Modifies a catalogue's metadata on the FDP server."""
    url = f"{fdp_url}/catalog/{catalogue_id}"
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-type': 'text/turtle'
    }
    response = requests.put(url=url, data=updated_catalog_data, headers=headers)
    return response.status_code, response.text

def modify_dataset(token, dataset_id, fdp_url, updated_dataset_data):
    """Modifies a dataset's metadata on the FDP server."""
    url = f"{fdp_url}/dataset/{dataset_id}"
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-type': 'text/turtle'
    }
    response = requests.put(url=url, data=updated_dataset_data, headers=headers)
    return response.status_code, response.text

# BELOW IS THE FUNCTION TO UPSERT A QUALITY CERTIFICATE SO WHEN THE USERS CLICK ON "UPDATE QUALITY CERTIFICATE" THE FUNCTION WILL BE CALLED AND CHECKS IF THE CERTIFICATE EXISTS OR NOT

def upsert_quality_certificate(token, dataset_uri, fdp_url, certificate_data):
    """
    Checks if a quality certificate exists for the dataset. If it exists, performs a PUT request to update it.
    Otherwise, performs a POST request to create a new quality certificate.

    Parameters:
    - token: The authentication token for FDP.
    - dataset_uri: URI of the dataset to check for the quality certificate. !!!!!
    - fdp_url: Base URL of the Fair Data Point (FDP).
    - certificate_data: RDF data in Turtle format for the quality certificate.
    
    Returns:
    - Tuple (status_code, response_text)
    """

    dataset_uri = "http://his-fdp-srv-t1.darwinproject.be:8080/dataset/f19bafa6-a58b-4e11-b504-55a613f000cc"
    # SPARQL query to check for existing quality certificates
    query = """
        PREFIX dqv: <http://www.w3.org/ns/dqv#>  
        SELECT ?certificates
        WHERE {
            ?p dqv:hasQualityAnnotation ?certificates .
        }
    """

    # Parse the dataset URI to check if a quality certificate exists
    g = Graph()
    g.parse(dataset_uri)

    # Execute the SPARQL query
    certificate_exists = False
    certificate_id = None

    for r in g.query(query):
        certificate_id = r["certificates"]
        certificate_exists = True
        break

    # Define headers for the request
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-type': 'text/turtle'
    }

    # If certificate exists, perform a PUT request to update it
    if certificate_exists:
        # Update the certificate
        url = certificate_id
        response = requests.put(url, data=certificate_data, headers=headers)
    else:
        # If no certificate exists, perform a POST request to create a new certificate
        url = f"{fdp_url}/hasQualityAnnotation"
        response = requests.post(url, data=certificate_data, headers=headers)

    return response.status_code, response.text


the below is the code to download the label so this code can be used in the button "dowload the RDF". 
! what we need to make it work is the dataset URI

In [ ]:
dataset_uri = "http://his-fdp-srv-t1.darwinproject.be:8080/dataset/f19bafa6-a58b-4e11-b504-55a613f000cc"

q = """
    PREFIX dqv: <http://www.w3.org/ns/dqv#>  

    SELECT ?certificates
    WHERE {
        ?p dqv:hasQualityAnnotation ?certificates .
	}

"""

# Create a Graph (subject, predicate, object), parse from the records
g = Graph().parse(dataset_uri)
certificate = ''
for r in g.query(q):
    print(r["certificates"])
    certificate = r["certificates"]

url = certificate
headers={'Accept':'text/turtle'}

res=requests.get(url=url, headers=headers)

print("Response content:", res.text)

# Save response content as a .ttl file
with open("output.ttl", "w", encoding="utf-8") as file:
    file.write(res.text)

print("File saved as certificate.ttl")